In [8]:
using JLD2
using FileIO
using DataFrames
using Dates
using Random
using HDF5  
using Statistics
using Interact
using Plots
base_data_path = "/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep"

"/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep"

In [2]:
function build_index_from_paths(base_path::String)
    rows = []
    println("📂 Scanning directory structure in: $base_path")

    # הגדרת ביטויים רגולריים (Regex) לחילוץ המספרים מהשם
    # הסבר: מחפשים את הטקסט הספציפי ואחריו קבוצה של ספרות ונקודות
    re_beta = r"beta_([\d\.]+)"
    re_h    = r"_h_([\d\.]+)"     # _h_ כדי לוודא שלא תופסים h במקומות אחרים
    re_eps  = r"eps_([\d\.]+)"
    re_ver  = r"_v(\d+)"          # תופס את המספר אחרי v

    for (root, dirs, files) in walkdir(base_path)
        # תנאי סף: אנחנו מתייחסים לתיקייה כסימולציה רק אם יש בה params.jld2
        # (אפילו שאנחנו לא קוראים אותו, זה מסמן שזו תיקיית ריצה ולא תיקיית ביניים)
        if "params.jld2" in files
            
            # 1. ניתוח הנתיב כדי להוציא את שם הפרויקט
            rel = relpath(root, base_path)
            path_parts = splitpath(rel)
            
            # המבנה הוא: Project / Server / SimFolder
            # לכן אנחנו צריכים שלפחות יהיו 2 חלקים (פרויקט ותיקיית סימולציה)
            if length(path_parts) >= 2
                folder_name = path_parts[end] # השם: beta_16.00_...
                project_name = path_parts[1]  # השם: psl_2_4
                
                # אתחול השורה
                row = Dict{Symbol, Any}()
                row[:project] = project_name
                row[:full_path] = root
                
                # 2. חילוץ פרמטרים משם התיקייה באמצעות Regex
                
                # Beta
                m_beta = match(re_beta, folder_name)
                row[:beta] = isnothing(m_beta) ? missing : parse(Float64, m_beta.captures[1])
                
                # h
                m_h = match(re_h, folder_name)
                row[:h] = isnothing(m_h) ? missing : parse(Float64, m_h.captures[1])
                
                # Epsilon
                m_eps = match(re_eps, folder_name)
                row[:epsilon] = isnothing(m_eps) ? missing : parse(Float64, m_eps.captures[1])
                
                # Version (v1 -> 1)
                m_ver = match(re_ver, folder_name)
                row[:version] = isnothing(m_ver) ? missing : parse(Int, m_ver.captures[1])
                
                push!(rows, row)
            end
        end
    end

    println("✅ Found $(length(rows)) simulations.")

    if isempty(rows)
        return DataFrame()
    end

    # יצירת ה-DataFrame
    df = DataFrame(rows)

    # סידור העמודות בסדר שביקשת
    desired_order = [:project, :beta, :h, :epsilon, :version, :full_path]
    
    # (מוודאים שכל העמודות קיימות כדי למנוע שגיאה אם התיקייה ריקה מתוצאות)
    final_cols = filter(c -> c in propertynames(df), desired_order)
    select!(df, final_cols)
    
    # מיון התוצאות (אופציונלי - נוח לעין)
    sort!(df, [:project, :beta, :h, :epsilon, :version])
    
    return df
end

build_index_from_paths (generic function with 1 method)

In [3]:
function get_running_avg(vec::Vector{Float64})
    return cumsum(vec) ./ (1:length(vec))
end

get_running_avg (generic function with 1 method)

In [4]:
function interactive_flux_explorer(df)

    gdf = groupby(df, [:beta, :h, :epsilon])    
    labels = String[]
    keys_list = []
    
    for key in keys(gdf)
        label_str = "β=$(key.beta), h=$(key.h), ε=$(key.epsilon)"
        push!(labels, label_str)
        push!(keys_list, key)
    end
    options = Dict(labels[i] => keys_list[i] for i in 1:length(labels))
    
    ui = @manipulate for params_key in options
        
        sub_df = gdf[params_key]
        
        p = plot(title="Spatial Flux Running Avg\n$(params_key)", 
                 xlabel="Sweep / Time Step", 
                 ylabel="Flux <W>",
                 legend=:bottomright)
        
        for row in eachrow(sub_df)
            meas_path = joinpath(row.full_path, "measurements.jld2")
            
            if isfile(meas_path)
                try
                    data = load(meas_path)
                    
                    if haskey(data, "measurements")
                        m = data["measurements"]
                        
                        if hasproperty(m, :avg_spatial_flux)
                            flux_vec = m.avg_spatial_flux
                            
                            y_vals = get_running_avg(flux_vec)
                            
                            plot!(p, y_vals, label="v$(row.version)", lw=2, alpha=0.8)
                        else
                            println("⚠️ Missing avg_spatial_flux in $(row.full_path)")
                        end
                    end
                catch e
                    println("❌ Error loading $meas_path: $e")
                end
            end
        end
        p
    end
    return ui
end

interactive_flux_explorer (generic function with 1 method)

In [9]:
function save_plots_to_pdf(df, filename="avg_flux_conv.pdf")
    gdf = groupby(df, [:beta, :h, :epsilon])
    
    # פתיחת אנימציה שתשמש כ-PDF
    anim = @animate for key in keys(gdf)
        sub_df = gdf[key]
        p = plot(title="β=$(key.beta), h=$(key.h), ε=$(key.epsilon)", legend=:bottomright)
        
        for row in eachrow(sub_df)
            path = joinpath(row.full_path, "measurements.jld2")
            if isfile(path)
                try
                    m = load(path)["measurements"]
                    y = cumsum(m.avg_spatial_flux) ./ (1:length(m.avg_spatial_flux))
                    plot!(p, y, label="v$(row.version)")
                catch; end
            end
        end
        # הגרף הנוכחי מתווסף לפריים
        plot!(p) 
    end
    
    # שמירה
    gif(anim, filename, fps=1) # טריק לשמירה, אבל עדיף savefig בלולאה רגילה אם זה PDF
    println("Saved report to $filename")
end

save_plots_to_pdf (generic function with 2 methods)

In [10]:
df = build_index_from_paths(base_data_path)

📂 Scanning directory structure in: /media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep


✅ Found 320 simulations.


Row,project,beta,h,epsilon,version,full_path
,String,Float64,Float64,Float64,Int64,String
1,psl_2_4,2.0,0.1,0.0156,1,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.0156_v1
2,psl_2_4,2.0,0.1,0.0156,2,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.0156_v2
3,psl_2_4,2.0,0.1,0.0312,1,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.0312_v1
4,psl_2_4,2.0,0.1,0.0312,2,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.0312_v2
5,psl_2_4,2.0,0.1,0.0625,1,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.0625_v1
6,psl_2_4,2.0,0.1,0.0625,2,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.0625_v2
7,psl_2_4,2.0,0.1,0.125,1,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.1250_v1
8,psl_2_4,2.0,0.1,0.125,2,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.10_eps_0.1250_v2
9,psl_2_4,2.0,0.2,0.0156,1,/media/omerpaz/3AEADFAA9E4DA17A/CSS_spin_rep/psl_2_4/landau/beta_2.00_h_0.20_eps_0.0156_v1


In [15]:
using Plots
using Statistics
using JLD2
using FileIO
using DataFrames

# שימוש ב-backend סטנדרטי ליצירת קבצים
gr()

# 1. פונקציית עזר לממוצע רץ
function get_running_avg(vec::Vector{Float64})
    return cumsum(vec) ./ (1:length(vec))
end

function save_all_graphs_as_images(df, output_dir="graphs_output_images")
    # יצירת התיקייה אם אינה קיימת (שיניתי מעט את השם שלא ידרוס את ה-PDFים אם יש)
    if !isdir(output_dir)
        mkdir(output_dir)
        println("📂 Created directory: $output_dir")
    end

    println("🚀 Starting batch image generation...")

    # 2. קיבוץ הנתונים לפי הפרמטרים הפיזיקליים
    gdf = groupby(df, [:beta, :h, :epsilon])
    
    total_groups = length(gdf)
    current = 0

    for key in keys(gdf)
        current += 1
        sub_df = gdf[key]
        
        # מיון לפי גרסה כדי שהמקרא יהיה מסודר
        sort!(sub_df, :version)
        
        # 3. יצירת הכותרת ושם הקובץ
        title_str = "Flux Running Avg\nβ=$(key.beta), h=$(key.h), ε=$(key.epsilon)"
        
        # --- השינוי היחיד: הסיומת היא png במקום pdf ---
        filename = "flux_beta_$(key.beta)_h_$(key.h)_eps_$(key.epsilon).png"
        full_save_path = joinpath(output_dir, filename)
        
        # אתחול הגרף (הגדלתי מעט את הרזולוציה שתראה טוב בתמונה)
        p = plot(title=title_str, 
                 xlabel="Sweep", 
                 ylabel="<W>", 
                 legend=:bottomright,
                 size=(1000, 750), dpi=100) # dpi גבוה יותר לתמונה חדה
        
        # 4. הוספת הקווים (לכל גרסה שיש בטבלה)
        lines_added = 0
        for row in eachrow(sub_df)
            meas_path = joinpath(row.full_path, "measurements.jld2")
            
            if isfile(meas_path)
                try
                    # טעינה
                    data = load(meas_path)
                    if haskey(data, "measurements")
                        m = data["measurements"]
                        # וידוא שהשדה קיים ושהוא וקטור
                        if hasproperty(m, :avg_spatial_flux)
                            flux_vec = m.avg_spatial_flux
                            y_vals = get_running_avg(flux_vec)
                            
                            plot!(p, y_vals, label="v$(row.version)", lw=2, alpha=0.8)
                            lines_added += 1
                        end
                    end
                catch e
                    # מתעלמים מקבצים שבורים נקודתית
                end
            end
        end
        
        # 5. שמירה לקובץ (רק אם באמת הוספנו קווים)
        if lines_added > 0
            savefig(p, full_save_path)
            # הדפסה כל 10 גרפים כדי לדעת שהסקריפט חי
            if current % 10 == 0
                println("   Saved $current / $total_groups images...")
            end
        end
    end
    
    println("✅ Done! All images saved in folder: $output_dir")
end

# --- הרצה ---
save_all_graphs_as_images(df)

📂 Created directory: graphs_output_images
🚀 Starting batch image generation...


┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in worksp

   Saved 10 / 160 images...


┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in worksp

   Saved 20 / 160 images...


...

┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in worksp

   Saved 150 / 160 images...


┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in worksp

   Saved 160 / 160 images...
✅ Done! All images saved in folder: graphs_output_images


┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.Measurements does not exist in workspace; reconstructing
└ @ JLD2 ~/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588


In [1]:
using Statistics
using JLD2
using FileIO
using DataFrames
using Plots

# 1. פונקציית השגיאה שביקשת (Binning Analysis)
function bin_analysis(data, min_sample_size=128)
    data_size = length(data)
    # מוודאים שיש מספיק נתונים
    if data_size < min_sample_size
        return [0.0] # או NaN, טיפול במקרה קצה
    end
    
    chopped_data_size = 2^floor(Int, log(2, data_size))
    chopped_data = collect(Iterators.take(Iterators.reverse(data), chopped_data_size))
    
    stds = [std(chopped_data) / sqrt(chopped_data_size)]
    bin_size = 2
    while min_sample_size < div(chopped_data_size, bin_size)
        length_bin = div(chopped_data_size, bin_size)
        binned = reshape(chopped_data, (bin_size, length_bin))
        means = vec(mean(binned, dims=1))
        std_bin = std(means) / sqrt(length_bin)
        
        bin_size *= 2
        push!(stds, std_bin)
    end
    return stds
end

# 2. שלב העיבוד: הוספת עמודות ממוצע ושגיאה ל-DataFrame
function process_statistics!(df)
    println("⚙️ Calculating means and errors for all simulations...")
    
    # הוספת עמודות ריקות אם הן לא קיימות
    if !("mean_flux" in names(df))
        df.mean_flux = Vector{Union{Float64, Missing}}(missing, nrow(df))
        df.error_flux = Vector{Union{Float64, Missing}}(missing, nrow(df))
    end

    total = nrow(df)
    for (i, row) in enumerate(eachrow(df))
        # אם כבר חישבנו, נדלג (חוסך זמן אם מריצים שוב)
        if !ismissing(row.mean_flux) continue end
        
        meas_path = joinpath(row.full_path, "measurements.jld2")
        if isfile(meas_path)
            try
                data = load(meas_path)
                if haskey(data, "measurements")
                    m = data["measurements"]
                    if hasproperty(m, :avg_spatial_flux)
                        flux_vec = m.avg_spatial_flux
                        
                        # חישוב הממוצע
                        row.mean_flux = mean(flux_vec)
                        
                        # חישוב השגיאה לפי bin_analysis
                        stds = bin_analysis(flux_vec)
                        row.error_flux = stds[end] # הטובין שבסוף
                    end
                end
            catch e
                # במקרה של שגיאה נשאיר missing
            end
        end
        
        if i % 50 == 0
            println("   Processed $i / $total simulations...")
        end
    end
    println("✅ Statistics calculation complete.")
end

# 3. שלב הציור: גרף לכל Beta
function plot_phase_diagrams(df, output_dir="flux vs h")
    if !isdir(output_dir)
        mkdir(output_dir)
    end

    # מסננים שורות שלא הצלחנו לחשב להן סטטיסטיקה
    valid_df = filter(row -> !ismissing(row.mean_flux), df)
    
    # קיבוץ לפי Beta (כי לכל בטא יהיה גרף נפרד)
    grouped_by_beta = groupby(valid_df, :beta)

    println("🎨 Generating plots...")
    
    for sub_df_beta in grouped_by_beta
        current_beta = sub_df_beta.beta[1] # הערך של בטא בקבוצה זו
        
        # יצירת הגרף
        p = plot(title="β=$(current_beta)",
                 xlabel="Transverse Field (h)",
                 ylabel="Mean Spatial Flux",
                 legend=:bottomleft,
                 size=(900, 600), dpi=150)
        
        # בתוך הגרף, אנחנו רוצים צבע שונה לכל Epsilon
        # אז נקבוץ לפי Epsilon
        grouped_by_eps = groupby(sub_df_beta, :epsilon)
        
        for sub_df_eps in grouped_by_eps
            current_eps = sub_df_eps.epsilon[1]
            
            # ממיינים לפי h כדי שהקו יעבור יפה (אם רוצים קו) או סתם לסדר
            sort!(sub_df_eps, :h)
            
            # חילוץ הנתונים לציור
            xs = sub_df_eps.h
            ys = sub_df_eps.mean_flux
            yerrs = sub_df_eps.error_flux
            
            # ציור Scatter עם שגיאות
            # msw = marker stroke width (מסגרת לנקודה), ms = marker size
            scatter!(p, xs, ys, yerror=yerrs, 
                     label="ε=$(current_eps)", 
                     ms=4, msw=0.5, marker=:circle)
            
            # אופציונלי: הוספת קו דק ומקוקו לחיבור הנקודות להקלה על העין
            plot!(p, xs, ys, label="", style=:dash, alpha=0.3, color=palette(:default)[plot_color_index(sub_df_eps)])
        end
        
        # שמירה
        filename = "flux_vs_h_beta_$(current_beta).png"
        savefig(p, joinpath(output_dir, filename))
        println("   Saved: $filename")
    end
    println("✅ Done. All phase diagrams saved in $output_dir")
end

# פונקציית עזר קטנה לקבלת אותו צבע לקו ולנקודות (לא קריטי, סתם ליופי)
function plot_color_index(df_slice)
    # טריק מלוכלך לקבלת האינדקס הנוכחי של הצבעים, פשוט נסמוך על הדיפולט של Scatter
    return 1 
end

# --- הרצה ---
# 1. חישוב הנתונים (משנה את df במקום)
process_statistics!(df)

# 2. ציור הגרפים
plot_phase_diagrams(df)

UndefVarError: UndefVarError: `df` not defined in `Main`
Suggestion: check for spelling errors or missing imports.